In [68]:
import pandas as pd
import sqlalchemy
import math
import os, sys, copy, random, pickle, cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append("../")

%load_ext autoreload
%autoreload 2

from modules.ControlTracking import TrackFactory
from modules.ControlDisplay import Display
from modules.Interproc import DBInterface, GuiviewState

from modules.AnalysisHelpers import ( 
         exploreImgs, buildImgComparisonData, subprocEval)

from modules.EvalHelpers import EvalTracker, EvalDataset
from modules.ControlEval import EvalFactory
from modules.DataSchemas import ScoreSchema
from modules.EvalHelpers import OutcomeData, DFHelper

from modules.EvalHelpers import AggEval, DFHelper

from modules.EvalSuite import EvalSuite

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Use subproc with logging

In [69]:
tmp_db_path = 'data/misc/books/eval-module-4/evaldb.db'

vid_path = 'data/proc/tmp/dec14/output5.proc1.proc1.avi'

outcome_subproc = subprocEval(f_pathfn = vid_path
                              ,db_pathfn = tmp_db_path 
                              ,b_log = True
                              ,b_eval_log = True
                             )


START - running eval module...
[                            ]
-----------------------------
output db: data/misc/books/eval-module-4/evaldb.db
output tbl: outcome_dataframe
eval time: 6.164
outcome rows: 345 cols: 48 

num frames:                  345
obj enums scored/tracked:    0,1 / 0
num scored frames:           19 | 0,9,78...308,324,340
-------
agg_checkBaselineInsideTrack       0.42
agg_checkBothContainsOther         0.42
agg_checkEitherContainsOther       0.53
agg_checkTrackInsideBaseline       0.53
agg_checkTrackInsideBaselineRect   0.11
agg_checkTrackSuccess              1.00
agg_compareRadii                   2.32
agg_distanceFromBaseline           5.49
subproc time: 30.43


### Use EvalSuite

In [70]:
from modules.EvalSuite import EvalSuite

In [71]:
suite = EvalSuite()
suite.buildFromOutcome(outcome_subproc)
suite.displayCli()


num frames:                  345
obj enums scored/tracked:    0,1 / 0
num scored frames:           19 | 0,9,78...308,324,340
-------
agg_checkBaselineInsideTrack       0.42
agg_checkBothContainsOther         0.42
agg_checkEitherContainsOther       0.53
agg_checkTrackInsideBaseline       0.53
agg_checkTrackInsideBaselineRect   0.11
agg_checkTrackSuccess              1.00
agg_compareRadii                   2.32
agg_distanceFromBaseline           5.49


In [72]:
suite.previewEachDf()

,max,mean,median,min,std,sum
agg_checkBaselineInsideTrack,NaN,0.421053,NaN,NaN,NaN,NaN
agg_checkBothContainsOther,NaN,0.421053,NaN,NaN,NaN,NaN
agg_checkEitherContainsOther,NaN,0.526316,NaN,NaN,NaN,NaN


,check Both Contains Other,distance From Baseline,check Track Success,check Baseline Inside Track,check Either Contains Other,check Track Inside Baseline,check Track Inside Baseline Rect,compare Radii
0,True,2,True,True,True,True,False,6
1,None,nan,None,None,None,None,None,nan
2,None,nan,None,None,None,None,None,nan


,index,input _data0 _0,input _data0 _1,input _data0 _2,input _data0 _3,input _data1 _0,input _data1 _1,input _data1 _2,input _data1 _3,input _data2 _0,...,track _data3 _2,track _data3 _3,track _obj _exists _0,track _obj _exists _1,track _obj _exists _2,track _obj _exists _3,track _obj _type _0,track _obj _type _1,track _obj _type _2,track _obj _type _3
0,0,107.0,111.0,None,None,214.0,218.0,None,None,54.0,...,None,None,True,None,None,None,circle,None,None,None
1,1,NaN,NaN,None,None,NaN,NaN,None,None,NaN,...,None,None,True,None,None,None,circle,None,None,None
2,2,NaN,NaN,None,None,NaN,NaN,None,None,NaN,...,None,None,True,None,None,None,circle,None,None,None
